In [38]:
# The EMG data for training the biophysical model is a list of lists of numpy arrays of floats.
# The outer list contains the unique files included in training
#    - Each element of that list is all of the timesteps from a single file
#        - Each element of that list is a numpy array of floats representing the EMG data at a single timestep

# The angle data is represented similarly.

In [39]:
import numpy as np
np.set_printoptions(linewidth=1000)
import pandas as pd
import os
from scipy import interpolate
import pickle
import struct
import sys
sys.path.append('/home/haptix/haptix/haptix_controller/handsim/src')
from BesselFilter import BesselFilterArr

In [40]:
def interpolateData(emg, pos, Fs_in, Fs_out=60):
    # assert emg.shape[0] == pos.shape[0], 'Input data not the same size - check me!!'
    if not emg.shape[0] == pos.shape[0]:
        dataLen = min(emg.shape[0], pos.shape[0])
        emg = emg[:dataLen, :]
        pos = pos[:dataLen, :]

    dataLen = emg.shape[0]
    t = np.arange(dataLen)/Fs_in
    upSampleRatio = Fs_out/Fs_in
    tOut = np.arange(dataLen*upSampleRatio)/Fs_out

    femgInterp = interpolate.interp1d(t, emg, axis=0, fill_value='extrapolate')
    fposInterp = interpolate.interp1d(t, pos, axis=0, fill_value='extrapolate')

    outputEMG = femgInterp(tOut)
    outputPos = fposInterp(tOut)

    return (outputEMG, outputPos)

def smoothData(pos, Fs=60, timeAverage=0.25):
    """Smooth out pos data over time using a timeAverage second window"""
    dataPointsSmoothing = int(timeAverage*Fs/4)
    numJoints = pos.shape[1]
    # numDataPoints = pos.shape[0]

    outputPos = np.zeros_like(pos)
    for i in range(numJoints):
        outputPos[:, i] = np.convolve(pos[:, i], np.concatenate(([0]*dataPointsSmoothing, [1]*(3*dataPointsSmoothing))), 'same')/(3*dataPointsSmoothing)

    return outputPos

def besselFilter(pos, Fs, numChannels=1, order=4, critFreqs=5, pad=1):
    # pad the data at the beginning for a full second to handle filter transient
    padLen = int(pad*Fs)
    padding = np.stack([pos[:, 0]]*padLen, axis=1)
    posFilter = np.concatenate([padding, pos], axis=1)

    filt = BesselFilterArr(numChannels=numChannels, order=order, critFreqs=critFreqs, fs=Fs, filtType='lowpass')
    filtered = filt.filter(posFilter)
    return filtered[:, padLen:] # remove the padding

In [110]:
class EMGintegrator():
    def __init__(self, usedChannels=None, noise=None, norms=None, outputFreq=60):
        self.outputFreq = outputFreq
        self.numElectrodes = 16 if usedChannels is None else len(usedChannels)
        self.usedChannels = usedChannels
        self.noise = noise
        self.norms = norms

    def initFilters(self):
        self.powerLineFilters = BesselFilterArr(numChannels=self.numElectrodes, order=8, critFreqs=[58, 62], fs=self.samplingFreq, filtType='bandstop') # remove power line noise and multiples up to 600 Hz
        self.highPassFilters = BesselFilterArr(numChannels=self.numElectrodes, order=4, critFreqs=20, fs=self.samplingFreq, filtType='highpass') # high pass removes motion artifacts and drift
        self.lowPassFilters = BesselFilterArr(numChannels=self.numElectrodes, order=4, critFreqs=3, fs=self.samplingFreq, filtType='lowpass') # smooth the envelope, when not using 'actually' integrated EMG

    def resetFilters(self):
        self.powerLineFilters.resetFilters()
        self.highPassFilters.resetFilters()
        self.lowPassFilters.resetFilters()

    def calcSampleFreq(self, timestamps):
        # remember that times are recorded in microseconds!
        dt = np.diff(timestamps) # here we scale back to get seconds from microseconds
        meanDt = np.mean(dt)
        # FOR SPECIAL CASE HERE
        self.outputFreq = round(1/meanDt)
        return meanDt, round(1/meanDt)

    def loadEMG(self, emgFile, emgTimeStamps):
        # need to load in the EMG as an [numElectrodes x numDataPoints] array due to the filter structure
        data = np.load(emgFile).T
        data = np.nan_to_num(data)
        timestamps = np.load(emgTimeStamps)
        (self.dt, self.samplingFreq) = self.calcSampleFreq(timestamps)

        self.rawEMG = data[:self.numElectrodes, :] if self.usedChannels is None else data[self.usedChannels, :]

        self.packetLen = int(np.ceil(self.samplingFreq/self.outputFreq))

    def padEMG(self, pad=1):
        # pad the start of the EMG file to remove the large spike post-filtering
        # pad is number of packets to pad with
        lenPad = int(pad*self.outputFreq*self.packetLen)

        padding = np.multiply(np.ones((lenPad, self.numElectrodes)), self.rawEMG[:, 1].T).T
        paddedEMG = np.append(padding, self.rawEMG, axis=1)

        return paddedEMG, lenPad

    def processEMGPacket(self, emgPacket):
        powerLine = self.powerLineFilters.filter(emgPacket)
        highPass = self.highPassFilters.filter(emgPacket)
        rect = np.abs(highPass)
        clip = np.clip(rect - self.noise[:, None], 0, None)

        lowPass = self.lowPassFilters.filter(clip)
        iEMG = np.clip(lowPass, 0, None)

        return np.asarray(iEMG)[:, -1]

    def integrateEMG(self):
        self.dataLen = self.rawEMG.shape[1]

        (paddedEMG, pad) = self.padEMG(10)
        numPackets = int(np.floor(self.dataLen/self.packetLen) + pad/self.packetLen)
        intEMG = np.zeros((numPackets, self.numElectrodes))

        # # highpass, powerline, rectify
        # powerLine = self.powerLineFilters.filter(paddedEMG)
        # highPass = self.highPassFilters.filter(powerLine)

        # rect = np.abs(highPass)

        for i in range(numPackets):
            # emgPacket = rect[:, i*self.packetLen:(i + 1)*self.packetLen]
            emgPacket = paddedEMG[:, i*self.packetLen:(i + 1)*self.packetLen]

            intEMG[i, :] = self.processEMGPacket(emgPacket=emgPacket)

        if not self.dataLen == numPackets*self.packetLen - pad:
            # lastiEMG = self.processEMGPacket(rect[:, numPackets*self.packetLen:])
            lastiEMG = self.processEMGPacket(paddedEMG[:, numPackets*self.packetLen:])

            intEMG = np.append(intEMG, lastiEMG[None, :], axis=0)

        # remove the padding
        self.intEMG = intEMG[int(pad/self.packetLen):, :]

    def normEMG(self):
        # self.normedEMG = np.divide(self.intEMG, self.norms)
        self.normedEMG = np.divide(self.intEMG, 1.33*np.max(self.intEMG[1000:], axis=0))

    def saveData(self, saveFol):
        # np.save(os.path.join(saveFol, 'intEMG.npy'), self.intEMG)
        # np.save(os.path.join(saveFol, 'normedEMG.npy'), self.normedEMG)
        np.save(os.path.join(saveFol, 'rawEMG.npy'), self.rawEMG.T)
        np.save(os.path.join(saveFol, 'intEmg.npy'), self.normedEMG)

    def processingPipeline(self, folName, saveFol):
        emgFile = os.path.join(folName, 'emg.npy')
        emgTimeStamps = os.path.join(folName, 'emg_timestamps.npy')
        self.loadEMG(emgFile=emgFile, emgTimeStamps=emgTimeStamps)
        self.initFilters()
        self.integrateEMG()
        self.normEMG()
        if saveFol is not None: self.saveData(saveFol=saveFol)

In [111]:
# values for P7
usedChannels = [0, 1, 2, 4, 5, 8, 10, 12]
usedAngles = ['thumbOutPlaneAng', 'indexAng', 'midAng', 'wristFlex']

# load the normalization factors
with open('/home/haptix/haptix/haptix_controller/handsim/include/scaleFactors.txt', 'rb') as f:
    normsPacked = f.read()
    norms = struct.unpack(32*'f', normsPacked)

    noise = np.array(norms[16:])
    norms = np.array(norms[:16])

# for some files, we recorded the raw data rather than normed and integrated, so we need to pre-preprocess them
baseFolder = '/home/haptix/UE AMI Clinical Work/Patient Data/P7 - 453/P7_0325_2024/Data Recording/'
recordingFolder = 'P7_0325_pickPlace'

integrator = EMGintegrator(usedChannels=usedChannels, noise=noise[usedChannels], norms=norms[usedChannels])
integrator.processingPipeline(folName=os.path.join(baseFolder, recordingFolder), saveFol=os.path.join(baseFolder, recordingFolder))


In [ ]:
# format the data